In [1]:
import socket
import pyaudio
import bluetooth, subprocess

import speech_recognition as sr
import pyttsx3
import time


import sounddevice as sd
import numpy as np
import scipy.signal
import timeit
import python_speech_features
import tensorflow as tf

In [2]:
#import socket
name = 'HC-05'      # Device name
addr = '34:03:de:33:f8:9d'      # Device Address
port = 3         # RFCOMM port 
 
nearby_devices = bluetooth.discover_devices(duration=5, lookup_names=True)
print('found %d devices' %len(nearby_devices))
 
for addr, name in nearby_devices:
    print('  %s - %s' %(addr, name))

found 3 devices
  FC:58:FA:B0:81:8F - TaoTronics SoundSurge 85
  41:42:8A:B7:FE:2A - BT
  95:05:BB:28:09:04 - Soundcore Life Q20


In [3]:
serverMACAddress = '00:21:09:00:31:30'  # Put your HC-05 address here
port = 1  # Match the setting on the HC-05 module
s = socket.socket(socket.AF_BLUETOOTH, socket.SOCK_STREAM, socket.BTPROTO_RFCOMM)
s.connect((serverMACAddress,port))

TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

In [ ]:
def send_bl(text):
    s.send(bytes(text, 'UTF-8'))
    s.send(bytes(text, 'UTF-8'))

# Speech-to-text API

In [ ]:
def give_com(MyText):
    if MyText=='on':
        send_bl('ON')
        send_bl('ON')

    elif MyText=='off' or MyText=='of':
        send_bl('OF')
        send_bl('OF')
    elif MyText=='go':
        send_bl('GO')
        send_bl('GO')
    elif MyText=='up' or MyText=='ub':
        send_bl('UP')
        send_bl('UP')
    elif MyText=='down' or MyText=='dawn':
        send_bl('DN')
        send_bl('DN')

    else:
        send_bl('S')

In [4]:
r = sr.Recognizer()
m=sr.Microphone()

In [ ]:
while(1):
        with m as source2:
            print('Please Say Something:')
            t=time.time()
            r.adjust_for_ambient_noise(source2, duration=0.8)
            while 1:
                try:
                    audio2=r.record(source2, duration=4)
                    MyText = r.recognize_google(audio2)
                    MyText = MyText.lower()
                    break
                except:
                    pass

            print("You said "+MyText+"   ##############")

Please Say Something:
You said on   ##############
Please Say Something:
You said off   ##############
Please Say Something:
You said tell me the news   ##############
Please Say Something:
You said the gallop   ##############
Please Say Something:
You said set an alarm   ##############
Please Say Something:


## Connect The Output

In [ ]:
debug_time = 1
debug_acc = 0
led_pin = 8
word_threshold = 0.5
rec_duration = 0.5
window_stride = 2
sample_rate = 48000
resample_rate = 8000
num_channels = 1
num_mfcc = 16
model_path = 'speech_model.h5'
model=tf.keras.models.load_model(model_path)
window = np.zeros(int(rec_duration * resample_rate) * 2)

def decimate(signal, old_fs, new_fs):
    if new_fs > old_fs:
        print("Error: target sample rate higher than original")
        return signal, old_fs    
    dec_factor = old_fs / new_fs
    if not dec_factor.is_integer():
        print("Error: can only decimate by integer factor")
        return signal, old_fs
    resampled_signal = scipy.signal.decimate(signal, int(dec_factor))
    return resampled_signal, new_fs


def sd_callback(rec, frames, time, status):
    
    start = timeit.default_timer()    
    sd.play(rec)

    
    if status:
        print('Error:', status)

    rec = np.squeeze(rec)
    rec, new_fs = decimate(rec, sample_rate, resample_rate)    
    window[:len(window)//2] = window[len(window)//2:]
    window[len(window)//2:] = rec
    mfccs = python_speech_features.base.mfcc(window, 
                                        samplerate=new_fs,
                                        winlen=0.256,
                                        winstep=0.050,
                                        numcep=num_mfcc,
                                        nfilt=26,
                                        nfft=2048,
                                        preemph=0.0,
                                        ceplifter=0,
                                        appendEnergy=False,
                                        winfunc=np.hanning)
    mfccs = mfccs.transpose()
    in_tensor = np.float32(mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1))
    preds=model.predict(in_tensor,verbose=False)
    print("You said "+np.argmax(preds.ravel()+"   ##############"))

In [ ]:
with sd.InputStream(channels=num_channels,
                    samplerate=sample_rate,
                    blocksize=int(sample_rate * rec_duration),
                    callback=sd_callback):
    while True:
        pass

In [ ]:
while(1):
        with m as source2:
            print('Please Say Something:')
            t=time.time()
            r.adjust_for_ambient_noise(source2, duration=0.8)
            while 1:
                try:
                    audio2=r.record(source2, duration=4)
                    print(audio2)
                    MyText = MyText.lower()
                    break
                except:
                    print('repeat')
                    
            time.sleep(0.3)
            print("You said "+MyText+"   ##############")

Please Say Something:
repeat
repeat
repeat
repeat
repeat
repeat
